In [ ]:
import os
import pandas as pd
new_dir = os.path.join('house_data','new_data.csv')
df = pd.read_csv(new_dir,index_col=0)

In [ ]:
## 尋找缺失值
%matplotlib inline
from function.graph import Report
Report(df).missing_plot()

In [ ]:
## 新增外部資料
import gc
f = os.path.join("output","forex.csv")
tw = os.path.join("output","TWSE_close.csv")
df = df.set_index("交易年月日")
df = df.sort_index()

df['forex'] = pd.read_csv(f,index_col=0)["forex"].reindex(df.index,method='ffill')
df['twse_stock_market'] = pd.read_csv(tw,index_col=0)['TWSE'].reindex(df.index,method='ffill')

#### 已操作完前面步驟
df.to_csv(new_dir)

In [ ]:
## 其他操作
## 文字資料轉換 dummy  & drop 掉建築完成年月 
data = df.drop(columns=['建築完成年月','土地位置建物門牌','wgs84'])
data.主要用途 = df.主要用途.fillna('其他')
data = pd.get_dummies(data)
data['wgs84']=df['wgs84']

## 匯出成analyze.csv
### 全部為0 : 主要建材_加強磚造 主要用途_其他 有無管理組織_無 車位類別_其他
### 交易標的_房地(土地+建物)+車位 建物型態_其他 建物現況格局-隔間_無 都市土地使用分區_其他
droplist = ["主要建材_加強磚造","主要用途_其他","交易標的_房地(土地+建物)+車位","建物型態_其他",
           "建物現況格局-隔間_無","有無管理組織_無","車位類別_其他","都市土地使用分區_其他",
            "鄉鎮市區_萬華區","電梯_無"]
data = data.drop(columns=droplist)
data.to_csv("analyze.csv")

In [ ]:
# 主要用途整併  
# 避免單一欄位過少
df = pd.read_csv('analyze.csv',index_col=0)

## 主要事務所欄位整併
import gc
df['主要用途_一般事務所'] = df['主要用途_一般事務所'].astype(int) + df['主要用途_第２８組一般事務所'].astype(int)+ df['主要用途_（第２８組）一般事務所'].astype(int)
df['主要用途_一般事務所_G2'] = df['主要用途_一般事務所（Ｇ－２）'].astype(int)+ df['主要用途_一般事務所（Ｇ２）'].astype(int)
df = df.drop(columns=['主要用途_第２８組一般事務所','主要用途_一般事務所（Ｇ－２）','主要用途_一般事務所（Ｇ２）','主要用途_（第２８組）一般事務所'])

## 零售業
df['主要用途_一般零售業甲組'] = df['主要用途_一般零售業甲組'].astype(int) + df['主要用途_一般零售業（甲組）'].astype(int) 
df = df.drop(columns=['主要用途_一般零售業（甲組）'])

df['主要用途_一般零售業甲組_不含日用百貨'] = df['主要用途_一般零售業甲組（G3）（不含日用百貨）'].astype(int) + df['主要用途_一般零售業甲組（不含便利商店使用）'].astype(int)
df['主要用途_一般零售業甲組_不含日用百貨'] = df['主要用途_一般零售業甲組_不含日用百貨'] + df['主要用途_一般零售業甲組（不含日用百貨業）'] + df['主要用途_一般零售業甲組（除便利商店以外）']
gc.collect()
df['主要用途_一般零售業甲組_不含日用百貨'] =df['主要用途_一般零售業甲組_不含日用百貨'] + df['主要用途_一般零售業（甲組）（不含便利商店）'] + df['主要用途_一般零售業（甲組）（不含日用百貨）']+ df['主要用途_一般零售業（甲組）Ｇ３不做便利商店使用']+ df['主要用途_一般零售業（甲組，排除日常百貨）']

df = df.drop(columns=['主要用途_一般零售業甲組（G3）（不含日用百貨）','主要用途_一般零售業甲組（不含便利商店使用）','主要用途_一般零售業甲組（不含日用百貨業）','主要用途_一般零售業甲組（除便利商店以外）','主要用途_一般零售業（甲組）（不含便利商店）',
                     '主要用途_一般零售業（甲組）（不含日用百貨）','主要用途_一般零售業（甲組）Ｇ３不做便利商店使用','主要用途_一般零售業（甲組，排除日常百貨）'])
gc.collect()


## 零售業 甲乙組
df['主要用途_一般零售業_甲乙組'] = df['主要用途_一般零售業甲組一般零售業乙組（日用百貨除外）'] + df['主要用途_一般零售業甲乙組，（Ｇ３）']
df = df.drop(columns=['主要用途_一般零售業甲組一般零售業乙組（日用百貨除外）','主要用途_一般零售業甲乙組，（Ｇ３）'])

## 多戶住宅
df['主要用途_多戶住宅'] = df['主要用途_多戶住宅']+df['主要用途_多戶住宅、住宅']+df['主要用途_多戶住宅、集合住宅']+df['主要用途_多戶住宅（第二組）']+ df['主要用途_多戶住宅（Ｈ２）集合住宅']+ df['主要用途_多戶住宅，（Ｈ２）集合住宅']+ df['主要用途_住宅，（Ｈ２）多戶住宅']
df['主要用途_多戶住宅'] = df['主要用途_多戶住宅'] + df['主要用途_多戶集合住宅'] + df['主要用途_第二組多戶住宅'] + df['主要用途_第二組多戶住宅（二）集合住宅'] + df['主要用途_第二組：多戶住宅（二）集合住宅（Ｈ２）']+ df['主要用途_第２組多戶住宅']
df = df.drop(columns=['主要用途_多戶住宅、住宅','主要用途_多戶住宅、集合住宅','主要用途_多戶住宅（第二組）','主要用途_多戶住宅（Ｈ２）集合住宅','主要用途_多戶住宅，（Ｈ２）集合住宅',
                     '主要用途_多戶集合住宅','主要用途_第二組多戶住宅','主要用途_第二組多戶住宅（二）集合住宅','主要用途_第二組：多戶住宅（二）集合住宅（Ｈ２）','主要用途_第２組多戶住宅'])
df['主要用途_多戶住宅'] =df['主要用途_多戶住宅'] + df['主要用途_集合住宅'] + df['主要用途_集合住宅、樓梯間、外廊、水箱、自有機械工具房、停車空間']+ df['主要用途_集合住宅（Ｈ２）']+df['主要用途_集合住宅，多戶住宅']+df['主要用途_（Ｈ２）集合住宅']
df['主要用途_多戶住宅'] =df['主要用途_多戶住宅']+df['主要用途_（Ｈ２）集合住宅，第二組：多戶住宅']+df['主要用途_Ｈ２多戶住宅']+ df['主要用途_Ｈ２集合住宅']
col = ['主要用途_集合住宅','主要用途_集合住宅、樓梯間、外廊、水箱、自有機械工具房、停車空間','主要用途_集合住宅（Ｈ２）','主要用途_集合住宅，多戶住宅','主要用途_（Ｈ２）集合住宅',
      '主要用途_（Ｈ２）集合住宅，第二組：多戶住宅','主要用途_Ｈ２多戶住宅','主要用途_Ｈ２集合住宅','主要用途_住宅，（Ｈ２）多戶住宅']
df = df.drop(columns=col)

del col
gc.collect()
## 策略性住宅
df['主要用途_策略型產業'] = df['主要用途_策略型產業、（供第一目：資訊服務業使用）']+df['主要用途_策略性產業〈資訊服務業〉']+ df['主要用途_策略性產業〈２８組資訊服務業〉']+ df['主要用途_策略性產業之資訊服務業事務所']+df['主要用途_策略性產業作資訊服務業，剪接錄音工作室行業之使用']
df['主要用途_策略型產業'] = df['主要用途_策略型產業']+ df['主要用途_策略性產業﹝資訊服務業﹞']+df['主要用途_策略性產業（產品展示服務業）']+ df['主要用途_策略性產業（資訊服務業、剪接錄音工作室）']+ df['主要用途_策略性產業（資訊服務業）']+df['主要用途_策略性產業（資訊服務業）一般辦公室']
df['主要用途_策略型產業'] = df['主要用途_策略型產業']+ df['主要用途_策略性產業（資訊服務業）／（Ｇ２）辦公場所']+df['主要用途_策略性產業（Ｇ２資訊服務業）']+df['主要用途_策略性產業（Ｇ２）（資訊服務業）']+df['主要用途_（Ｇ２）策略性產業']

col =['主要用途_策略型產業、（供第一目：資訊服務業使用）','主要用途_策略性產業〈資訊服務業〉','主要用途_策略性產業〈２８組資訊服務業〉','主要用途_策略性產業之資訊服務業事務所','主要用途_策略性產業作資訊服務業，剪接錄音工作室行業之使用',
     '主要用途_策略性產業﹝資訊服務業﹞','主要用途_策略性產業（產品展示服務業）','主要用途_策略性產業（資訊服務業、剪接錄音工作室）','主要用途_策略性產業（資訊服務業）','主要用途_策略性產業（資訊服務業）一般辦公室','主要用途_策略性產業（資訊服務業）／（Ｇ２）辦公場所',
     '主要用途_策略性產業（Ｇ２資訊服務業）','主要用途_策略性產業（Ｇ２）（資訊服務業）','主要用途_（Ｇ２）策略性產業']
df = df.drop(columns=col)
del col
gc.collect()

## 自由業事務所 # 辦公用
df['主要用途_自由業事務所'] = df['主要用途_自由業事務所']+df['主要用途_自由職業事務所']
df['主要用途_辦公室'] = df['主要用途_辦公室'] + df['主要用途_辦公用']
col = ['主要用途_自由職業事務所','主要用途_辦公用']
df = df.drop(columns=col)
del col
gc.collect()


## 都市土地使用分區整併 ##錯字、過細分類
df['都市土地使用分區_都市：其他:交通用地'] =df['都市土地使用分區_都市：其他:交通用地']+df['都市土地使用分區_都市：其他:交通用地-公共設施用地']
col = ['都市土地使用分區_都市：其他:交通用地-公共設施用地']
# col.extend(['1',2,3]) # output : ['都市土地使用分區_都市：其他:交通用地-公共設施用地', '1', 2, 3]
df['都市土地使用分區_都市：其他:住宅區'] = df['都市土地使用分區_都市：其他:住宅區']  + df['都市土地使用分區_都市：其他:住宅用地']
col.extend(['都市土地使用分區_都市：其他:住宅用地'])

df['都市土地使用分區_都市：其他:商業區'] = df['都市土地使用分區_都市：其他:商業區']+ df['都市土地使用分區_都市：其他:商業區 但是否在道路用地']
col.extend(['都市土地使用分區_都市：其他:商業區 但是否在道路用地'])

df['都市土地使用分區_都市：其他:特定觀光商業專用區'] = df['都市土地使用分區_都市：其他:特定觀光商業專用區']+ df['都市土地使用分區_都市：其他:特定觀光商業專用區 但是否在綠帶用地']+ df['都市土地使用分區_都市：其他:特定觀光商業專用區 但是否在道路用地']
col.extend(['都市土地使用分區_都市：其他:特定觀光商業專用區 但是否在道路用地','都市土地使用分區_都市：其他:特定觀光商業專用區 但是否在綠帶用地'])

df['都市土地使用分區_都市：其他:第一種住宅區'] = df['都市土地使用分區_都市：其他:第一種住宅區'] + df['都市土地使用分區_都市：其他:第一種住宅區 但是否在道路用地']
col.append('都市土地使用分區_都市：其他:第一種住宅區 但是否在道路用地')

## 異體字
df['都市土地使用分區_都市：其他:第三種商業區'] = df['都市土地使用分區_都市：其他:第三種商業區'] + df['都市土地使用分區_都市：其他:第參種商業區']
col.append('都市土地使用分區_都市：其他:第參種商業區')
df['都市土地使用分區_都市：其他:第四種商業區'] = df['都市土地使用分區_都市：其他:第四種商業區']+df['都市土地使用分區_都市：其他:第肆種商業區']
col.append('都市土地使用分區_都市：其他:第肆種商業區')

df['都市土地使用分區_都市：其他:聯合開發區'] = df['都市土地使用分區_都市：其他:聯合發開區'] + df['都市土地使用分區_都市：其他:聯合開發區']+ df['都市土地使用分區_都市：其他:連合開發區']
col.extend(['都市土地使用分區_都市：其他:聯合發開區','都市土地使用分區_都市：其他:連合開發區'])

df['都市土地使用分區_都市：其他:道路用地'] = df['都市土地使用分區_都市：其他:道路用地']+df['都市土地使用分區_都市：其他:道']

df = df.drop(columns=col)
gc.collect()

df[df['都市土地使用分區_都市：其他:第三之二種住宅區']!=0]
df.to_csv('analyze.csv')

In [ ]:
# 之後整合範本  ## save to new_data  ## analyze_data : add dummy and trans distance < 1 kilometer
# using map replace apply to boost 
import pandas as pd
import os
from package.distance import haversine,convert2lat_lon # greatCircleDistance
global hosp,sch,mrt,train,shop
hosp = pd.read_csv('output/hospital_final.csv',index_col=0)['wgs84'].map(convert2lat_lon)
sch = pd.read_csv('output/hospital_final.csv',index_col=0)['wgs84'].map(convert2lat_lon)
mrt = pd.read_csv('output/mrt_final.csv',index_col=0)['wgs84'].map(convert2lat_lon)
train = pd.read_csv('output/train_final.csv',index_col=0)['wgs84'].map(convert2lat_lon)
shop = pd.read_csv('output/shop_addr.csv',index_col=0,encoding='big5')['0'].map(convert2lat_lon)
main = pd.read_csv('house_data/analyze.csv',index_col=0)

# min school # 地址 (nan, nan) #47271 的部分
import numpy as np
def minimum_sch(s): # type(s)=tuple
    a1,a2=s
    if a1=='nan':
        return np.nan
    else:
        li = []
        for i in sch: # lat
            li.append(haversine(s,i)) # greatCircleDistance
        return min(li)
# min train
def minimum_train(s):
    a1,a2=s
    if a1=='nan':
        return np.nan
    else:
        li = []
        for sh in train:
            li.append(haversine(s,sh)) # greatCircleDistance
        return min(li)
def minimum_hosp(s):
    a1,a2 = s
    if a1=='nan':
        return np.nan
    else:
        li = []
        for sh in hosp:
            li.append(haversine(s,sh)) # greatCircleDistance
        return min(li)
def minimum_shop(s):
    a1,a2 = s
    if a1=='nan':
        return np.nan
    else:
        li = []
        for sh in shop:
            li.append(haversine(s,sh)) # greatCircleDistance
        return min(li)

def minimum_mrt(s):
    a1,a2 = s
    if a1=='nan':
        return np.nan
    else:
        li = []
        for i in mrt:
            li.append(haversine(s,i)) # greatCircleDistance
        return min(li)

main['wgs84'] = main['wgs84'].map(convert2lat_lon)

main['Min_school distance'] = main['wgs84'].map(minimum_sch)
main['Min_shopping_mall distance'] = main['wgs84'].map(minimum_shop)
main['Min_mrt distance'] = main['wgs84'].map(minimum_mrt)
main['Min_train_distance'] = main['wgs84'].map(minimum_train)
main['Min_hospital_distance']= main['wgs84'].map(minimum_hosp)

In [ ]:
## 處理交易年月日  
## 處理錯字
df = pd.read_csv('analyze.csv',index_col=0)
df.rename(columns={'屋齡byDat平方':'屋齡byDay平方'}, inplace = True)
df['交易年'] = df['交易年月日'].apply(lambda s : pd.to_datetime(s).year).astype(int)
df['交易月'] = df['交易年月日'].apply(lambda s : pd.to_datetime(s).month).astype(int)
df['交易日'] = df['交易年月日'].apply(lambda s : pd.to_datetime(s).day).astype(int)
df = df.drop(columns='交易年月日') 
df = df.drop(columns='wgs84')  ## 移除 wgs84 定位

## 建立 pro_analyze.csv
import numpy as np
col = ['forex','twse_stock_market','車位總價元','單價元平方公尺','總價元']
for i in col:
    df[i]=(np.log1p(df[i].dropna()))  # 把缺失值刪除，並把特定欄位取log轉換
    
df = df.dropna()
df.to_csv("pro_analyze.csv")